In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from pymongo import MongoClient
import plotly.graph_objects as go

In [2]:
# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["trust_analysis_db"]
collection = db["merged_trust_population"]

In [3]:
data = pd.DataFrame(list(collection.find({}, {
    "_id": 0,  # Excluded MongoDB's _id field
    "country": 1,
    "Latitude": 1,  
    "Longitude": 1, 
    "doctor_and_nurses": 1,
    "government": 1,
    "journalist": 1,
    "neighbourhood": 1,
    "philantropist": 1,
    "scientist": 1,
    "traditional_healers": 1,
    "population": 1
})))

In [4]:
# making sure lat/long columns are numeric
data["Latitude"] = pd.to_numeric(data["Latitude"], errors="coerce")
data["Longitude"] = pd.to_numeric(data["Longitude"], errors="coerce")


print(data.head())

     country  doctor_and_nurses  government  journalist  neighbourhood  \
0      China              90.86       56.93       74.62          88.08   
1      India              85.67       74.18       69.12          68.36   
2  Indonesia              81.09       70.82       63.41          80.38   
3    Nigeria              64.21       21.11       37.00          42.47   
4     Brazil              84.87       42.98       56.36          53.58   

   philantropist    population  scientist  traditional_healers  Latitude  \
0          73.24  1.426106e+09      96.57                92.47   39.9075   
1          65.68  1.402618e+09      85.30                38.20   28.6667   
2          58.64  2.748149e+08      70.11                66.59   -6.2118   
3          44.08  2.139962e+08      47.41                35.11    9.0574   
4          65.22  2.086608e+08      79.73                26.95  -15.7797   

   Longitude  
0   116.3972  
1    77.2167  
2   106.8416  
3     7.4898  
4   -47.9297  


In [5]:
trust_metrics = [
    "doctor_and_nurses", "government", "journalist",
    "neighbourhood", "philantropist", "scientist", "traditional_healers"
]

In [18]:
# world map
fig = go.Figure()

# added a scattergeo layer for government trust rates
fig.add_trace(go.Scattergeo(
    lat=data['Latitude'],
    lon=data['Longitude'],
    text=data['country'] + "<br>Trust in Government: " + data['government'].astype(str) + "%",
    mode='markers',
    marker=dict(
        size=10,  
        color=data['government'], 
        colorscale='hot',
        colorbar_title="Trust in Government (%)",
        line=dict(width=0.5, color='black'),
    )
))

# updated the map layout
fig.update_layout(
    title='Global Trust in Government by Country',
    geo=dict(
        showframe=True,
        framecolor='black',
        framewidth=2,
        showcoastlines=True,
        coastlinecolor="gray",
        showland=True,
        landcolor="lightgray",
        projection_type='natural earth',
        showcountries=True,
        countrycolor="black",  
        countrywidth=0.3
    ),
    template='plotly',
    height=800,
    width=1200
)

fig.show()